In [1]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null


In [2]:
%%writefile OnlineExamination.java
// File: OnlineExamination.java

import java.util.ArrayList;
import java.util.HashMap;
import java.util.List;
import java.util.Map;
import java.util.Scanner;
import java.util.Timer;
import java.util.TimerTask;

public class OnlineExamination {

    public static void main(String[] args) {
        Scanner scanner = new Scanner(System.in);
        AuthenticationService authService = new AuthenticationService();
        ProfileService profileService = new ProfileService();
        ExamService examService = new ExamService();

        System.out.println("--- Welcome to the Online Examination System ---");

        // Main application loop
        while (true) {
            // If no user is logged in, show the login prompt
            if (!SessionManager.isLoggedIn()) {
                System.out.println("\nPlease Login to Continue (or type 'exit' to close)");
                System.out.print("Enter username: ");
                String username = scanner.nextLine();

                if ("exit".equalsIgnoreCase(username)) {
                    System.out.println("Thank you for using the system. Goodbye! 👋");
                    break;
                }

                System.out.print("Enter password: ");
                String password = scanner.nextLine();

                User user = authService.login(username, password);
                if (user != null) {
                    SessionManager.login(user);
                }
            } else {
                // If a user is logged in, show the main menu
                System.out.println("\n--- Dashboard ---");
                System.out.println("1. Start Exam 📝");
                System.out.println("2. Update Profile and Password 👤");
                System.out.println("3. Logout 🚪");
                System.out.print("Choose an option: ");

                String choice = scanner.nextLine();

                switch (choice) {
                    case "1":
                        examService.startExam(scanner);
                        break;
                    case "2":
                        profileService.updateProfile(scanner);
                        break;
                    case "3":
                        SessionManager.logout();
                        break;
                    default:
                        System.out.println("Invalid option. Please try again.");
                        break;
                }
            }
        }
        scanner.close();
    }
}


// ===== SUPPORTING CLASSES =====

class User {
    private String username;
    private String password;
    private String fullName;

    public User(String username, String password, String fullName) {
        this.username = username;
        this.password = password;
        this.fullName = fullName;
    }

    public String getUsername() { return username; }
    public String getPassword() { return password; }
    public String getFullName() { return fullName; }

    public void setPassword(String password) { this.password = password; }
    public void setFullName(String fullName) { this.fullName = fullName; }
}

class Question {
    private String questionText;
    private String[] options;
    private int correctAnswerIndex; // 0-based index

    public Question(String questionText, String[] options, int correctAnswerIndex) {
        this.questionText = questionText;
        this.options = options;
        this.correctAnswerIndex = correctAnswerIndex;
    }

    public String getQuestionText() { return questionText; }
    public String[] getOptions() { return options; }
    public int getCorrectAnswerIndex() { return correctAnswerIndex; }
}

class SessionManager {
    private static User currentUser = null;

    public static void login(User user) { currentUser = user; }
    public static void logout() {
        System.out.println("You have been logged out.");
        currentUser = null;
    }
    public static User getCurrentUser() { return currentUser; }
    public static boolean isLoggedIn() { return currentUser != null; }
}

class AuthenticationService {
    private Map<String, User> userDatabase;

    public AuthenticationService() {
        userDatabase = new HashMap<>();
        // Pre-populate with sample users
        userDatabase.put("student1", new User("student1", "pass123", "Alice Smith"));
        userDatabase.put("student2", new User("student2", "pass456", "Bob Johnson"));
    }

    public User login(String username, String password) {
        User user = userDatabase.get(username);
        if (user != null && user.getPassword().equals(password)) {
            System.out.println("Login successful! Welcome, " + user.getFullName());
            return user;
        }
        System.out.println("Error: Invalid username or password.");
        return null;
    }
}

class ProfileService {
    public void updateProfile(Scanner scanner) {
        User currentUser = SessionManager.getCurrentUser();
        System.out.println("\n--- Update Profile ---");

        System.out.println("Current name: " + currentUser.getFullName());
        System.out.print("Enter new full name (or press Enter to keep current): ");
        String newName = scanner.nextLine();
        if (!newName.trim().isEmpty()) {
            currentUser.setFullName(newName);
            System.out.println("✅ Name updated successfully.");
        }

        System.out.print("Enter new password (or press Enter to keep current): ");
        String newPassword = scanner.nextLine();
        if (!newPassword.trim().isEmpty()) {
            currentUser.setPassword(newPassword);
            System.out.println("✅ Password updated successfully.");
        }
    }
}

class ExamService {
    private List<Question> questions;
    private volatile boolean timeUp = false; // 'volatile' to ensure visibility across threads
    private static final int EXAM_DURATION_SECONDS = 60; // 1 minute exam

    public ExamService() {
        questions = new ArrayList<>();
        questions.add(new Question("What is the capital of France?", new String[]{"1. Berlin", "2. Madrid", "3. Paris", "4. Rome"}, 2));
        questions.add(new Question("Which company developed Java?", new String[]{"1. Microsoft", "2. Sun Microsystems", "3. Apple", "4. Google"}, 1));
        questions.add(new Question("What is 2 + 2?", new String[]{"1. 3", "2. 4", "3. 5", "4. 6"}, 1));
        questions.add(new Question("Which planet is known as the Red Planet?", new String[]{"1. Earth", "2. Mars", "3. Jupiter", "4. Venus"}, 1));
    }

    public void startExam(Scanner scanner) {
        timeUp = false;
        int[] userAnswers = new int[questions.size()];
        java.util.Arrays.fill(userAnswers, -1); // -1 indicates unanswered

        System.out.println("\n--- Online Exam Started ---");
        System.out.println("You have " + EXAM_DURATION_SECONDS + " seconds. Good luck!");

        // Set up a timer for auto-submission
        Timer timer = new Timer();
        timer.schedule(new TimerTask() {
            @Override
            public void run() {
                timeUp = true;
            }
        }, EXAM_DURATION_SECONDS * 1000);

        // Loop through questions
        for (int i = 0; i < questions.size(); i++) {
            if (timeUp) {
                System.out.println("\n\nTime is up! Auto-submitting your answers...");
                break;
            }
            displayQuestion(i);

            // Get user input with a simple approach
            try {
                String input = scanner.nextLine();
                int answer = Integer.parseInt(input.trim());
                if (answer >= 1 && answer <= 4) {
                    userAnswers[i] = answer - 1; // Store 0-based index
                }
            } catch (NumberFormatException e) {
                System.out.println("Invalid input. Question marked as unanswered.");
            }
        }

        timer.cancel(); // Stop the timer
        submitAndCalculateScore(userAnswers);
    }

    private void displayQuestion(int questionIndex) {
        Question q = questions.get(questionIndex);
        System.out.println("\nQuestion " + (questionIndex + 1) + ": " + q.getQuestionText());
        for (String option : q.getOptions()) {
            System.out.println(option);
        }
        System.out.print("Your choice (1-4): ");
    }

    private void submitAndCalculateScore(int[] userAnswers) {
        System.out.println("\n--- Exam Finished ---");
        int score = 0;
        for (int i = 0; i < questions.size(); i++) {
            if (userAnswers[i] == questions.get(i).getCorrectAnswerIndex()) {
                score++;
            }
        }
        System.out.println("Your score: " + score + "/" + questions.size());
        System.out.println("Thank you for taking the exam, " + SessionManager.getCurrentUser().getFullName() + "!");
    }
}


Writing OnlineExamination.java


In [3]:
!javac OnlineExamination.java


In [4]:
!java OnlineExamination


--- Welcome to the Online Examination System ---

Please Login to Continue (or type 'exit' to close)
Enter username: student1
Enter password: pass123
Login successful! Welcome, Alice Smith

--- Dashboard ---
1. Start Exam 📝
2. Update Profile and Password 👤
3. Logout 🚪
Choose an option: 1

--- Online Exam Started ---
You have 60 seconds. Good luck!

Question 1: What is the capital of France?
1. Berlin
2. Madrid
3. Paris
4. Rome
Your choice (1-4): 3

Question 2: Which company developed Java?
1. Microsoft
2. Sun Microsystems
3. Apple
4. Google
Your choice (1-4): 2

Question 3: What is 2 + 2?
1. 3
2. 4
3. 5
4. 6
Your choice (1-4): 4

Question 4: Which planet is known as the Red Planet?
1. Earth
2. Mars
3. Jupiter
4. Venus
Your choice (1-4): 2

--- Exam Finished ---
Your score: 3/4
Thank you for taking the exam, Alice Smith!

--- Dashboard ---
1. Start Exam 📝
2. Update Profile and Password 👤
3. Logout 🚪
Choose an option: 2

--- Update Profile ---
Current name: Alice Smith
Enter new full nam